In [1]:
!pip install kaggle


In [32]:
from google.colab import files

# Upload kaggle.json file
uploaded = files.upload()


Saving kaggle.json to kaggle (1).json


In [33]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [34]:
!kaggle competitions download -c medical-specialty-classification

medical-specialty-classification.zip: Skipping, found more recently modified local copy (use --force to force download)


In [35]:
import zipfile
import os

# Replace 'target_directory' with the directory where you downloaded the dataset
with zipfile.ZipFile('/content/medical-specialty-classification.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/medical_data_NLP')

# List the contents of the extracted directory
os.listdir('/content/medical_data_NLP')


['Sample Submission.csv', 'Train.csv', 'test.csv']

In [36]:
import pandas as pd

# Replace 'file_path' with the path to your dataset file
train_df = pd.read_csv('/content/medical_data_NLP/Train.csv')
test_df= pd.read_csv('/content/medical_data_NLP/test.csv')
train_df['medical_specialty'] = train_df['medical_specialty'].str.replace('/', '-')
train_df.head()

,Id,description,medical_specialty,sample_name,transcription
0,1,Patient had a recurrent left arm pain after h...,Cardiovascular / Pulmonary,Angina - Consult,"HISTORY OF PRESENT ILLNESS: , The patient is a..."
1,2,The patient is an 84-year-old female presente...,Cardiovascular / Pulmonary,Pericardial Effusion,"REASON FOR CONSULTATION:, Pericardial effusio..."
2,3,Hand dermatitis.,Dermatology,Dermatitis - SOAP,"SUBJECTIVE:, This is a 29-year-old Vietnamese..."
3,4,Recurrent degenerative spondylolisthesis and ...,Surgery,Lumbar Re-exploration,"PREOPERATIVE DIAGNOSIS: , Recurrent degenerati..."
4,5,Chiropractic IME with old files review. Deta...,IME-QME-Work Comp etc.,Chiropractic IME - 2,"DATE OF INJURY : October 4, 2000,DATE OF EXAMI..."


In [65]:
from sklearn.model_selection import train_test_split

# Assuming your DataFrame is named 'df'
# Replace 'target_column' with the actual column used for classification
# and adjust the test_size and random_state as needed

# Separate features (X) and target (y)
X = train_df[['text']]  # Assuming 'text' is the column containing the input text data
y = train_df['medical_specialty']  # Assuming 'medical_specialty' is the target column

# Split the data into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

# Display the shapes of the resulting sets
print("Training set shape:", X_train.shape, y_train.shape)
print("Validation set shape:", X_valid.shape, y_valid.shape)


Training set shape: (2707, 1) (2707,)
Validation set shape: (677, 1) (677,)


35 different categories. For the base model, we will be using only the 'description' column for classification.

In [81]:
for i in X_valid['text']:
  print("Text is \n")
  print(i)

Text is 

 Possible cerebrovascular accident.  The EEG was obtained using 21 electrodes placed in scalp-to-scalp and scalp-to-vertex montages.  transcription: DIAGNOSIS:,  Possible cerebrovascular accident.,DESCRIPTION: , The EEG was obtained using 21 electrodes placed in scalp-to-scalp and scalp-to-vertex montages.  The background activity appears to consist of fairly organized somewhat pleomorphic low to occasional medium amplitude of 7-8 cycle per second activity and was seen mostly posteriorly bilaterally symmetrically.  A large amount of movement artifacts and electromyographic effects were noted intermixed throughout the recording session.  Transient periods of drowsiness occurred naturally producing irregular 5-7 cycle per second activity mostly over the anterior regions.  Hyperventilation was not performed.  No epileptiform activity or any definite lateralizing findings were seen.,IMPRESSION: , Mildly abnormal study.  The findings are suggestive of a generalized cerebral disord

In [72]:
y_train

2443        Consult - History and Phy.
1370        Consult - History and Phy.
2632                  Gastroenterology
839                         Orthopedic
3076                         Radiology
                     ...              
1095                           Surgery
1130             Hematology - Oncology
1294     SOAP - Chart - Progress Notes
860         Cardiovascular - Pulmonary
3174                           Surgery
Name: medical_specialty, Length: 2707, dtype: object

In [82]:
y_train.unique()

array([' Consult - History and Phy.', ' Gastroenterology', ' Orthopedic',
       ' Radiology', ' Discharge Summary', ' Nephrology',
       ' Cardiovascular - Pulmonary', ' Podiatry',
       ' ENT - Otolaryngology', ' Neurology', ' Dentistry',
       ' Neurosurgery', ' Surgery', ' SOAP - Chart - Progress Notes',
       ' Hematology - Oncology', ' Allergy - Immunology',
       ' Ophthalmology', ' General Medicine', ' IME-QME-Work Comp etc.',
       ' Office Notes', ' Pain Management', ' Chiropractic',
       ' Physical Medicine - Rehab', ' Dermatology', ' Endocrinology',
       ' Psychiatry - Psychology', ' Cosmetic - Plastic Surgery',
       ' Letters', ' Speech - Language', ' Rheumatology',
       ' Sleep Medicine', ' Bariatrics', ' Lab Medicine - Pathology',
       ' SCARF type', ' Autopsy'], dtype=object)

Making a ImageNet style folder structure for better FastAI integration

In [96]:
from google.colab import drive
import os
import pandas as pd

# Mount Google Drive
drive.mount('/content/drive')

# Assuming your DataFrame is named 'df' and the column is 'medical_specialty'



# Assuming your DataFrame is named 'df'
root_folder = "/content/drive/MyDrive/Kaggle_int_hc/train"  # Replace with your desired folder path
df_classes = y_train.unique()

# Create class folders
for class_name in df_classes:
    class_folder = os.path.join(root_folder, class_name)
    os.makedirs(class_folder, exist_ok=True)

# Save text data in class folders
for value, class_name in enumerate(zip(X_train['text'],y_train)):
    text_data = str(class_name[0])  # or 'transcription' depending on what you want to use
    class_name = class_name[1]
    filename = f"{value}.txt"  # Using 'Id' as a unique identifier
    file_path = os.path.join(root_folder, class_name, filename)

    with open(file_path, 'w', encoding='utf-8') as file:
      file.write(text_data)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [97]:
root_folder = "/content/drive/MyDrive/Kaggle_int_hc/valid"  # Replace with your desired folder path
df_classes = y_valid.unique()

# Create class folders
for class_name in df_classes:
    class_folder = os.path.join(root_folder, class_name)
    os.makedirs(class_folder, exist_ok=True)

# Save text data in class folders
for value, class_name in enumerate(zip(X_valid['text'],y_valid)):
    text_data = str(class_name[0])  # or 'transcription' depending on what you want to use
    class_name = class_name[1]
    filename = f"{value}.txt"  # Using 'Id' as a unique identifier
    file_path = os.path.join(root_folder, class_name, filename)

    with open(file_path, 'w', encoding='utf-8') as file:
      file.write(text_data)
